<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/231208-%E5%BE%97%E5%88%B0sample_2023%E7%9A%845d-NDRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=mIj46xMfb20Hzr9Nc211gbuya5aiWg9LZb0zp-Z3ZVw&tc=t7bKEZ0gqPSUb76xp5ac1ooLLhWppjcsrNNBv5-jdJY&cc=0sJ0pkwlbtLKCVeiJm9Nh6EArqYxGORajsez_s7Go1o

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnd2AE4dEfMAc5qGCnawC_MVlecgSrt2PEhQk9suALlMtGE50yD2jM

Successfully saved authorization token.


In [2]:
import os
import pandas as pd
import math

In [5]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
point = ee.FeatureCollection("projects/ee-grn/assets/sample_3class")

In [16]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2022-01-01','2023-12-08').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B5']).rename('NDRE')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

1276


In [17]:
# 二阶谐波
dataset = filteredSentinel.select(['NDRE','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDRE')

harmonicIndependents = ee.List(['constant', 't', 'cos1', 'sin1', 'cos2', 'sin2'])

def add_sincos(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  timeRadians2 = image.select('t').multiply(4 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1')).addBands(timeRadians2.cos().rename('cos2')).addBands(timeRadians2.sin().rename('sin2'))

harmonicSentinel = dataset.map(add_sincos)

harmonicTrend = harmonicSentinel.select(harmonicIndependents.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents.length(),numY=1))

harmonicTrendCoefficients = harmonicTrend.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents])

def fit(image):
  return image.addBands(image.select(harmonicIndependents).multiply(harmonicTrendCoefficients).reduce('sum').rename('fitted'))

fittedHarmonic = harmonicSentinel.map(fit)

print(fittedHarmonic.size().getInfo())

1276


In [18]:
# 5天序列NDRE
interval = 5
increment = 'day'
start = '2022-05-01'
startDate = ee.Date(start)
secondDate = startDate.advance(interval, increment).millis()
increase = secondDate.subtract(startDate.millis())
list5d = ee.List.sequence(startDate.millis(), ee.Date('2022-09-01').millis(), increase)

def add_time(date):
  return ee.Image(0).mask(0).rename('x').set('system:time_start',ee.Date(date).millis())

dailyMean =  ee.ImageCollection.fromImages(list5d.map(add_time))

def add_allbands(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  timeRadians1 = ee.Image(years).multiply(2 * math.pi)
  timeRadians2 = ee.Image(years).multiply(4 * math.pi)
  return image.addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1)).addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1')).addBands(timeRadians2.cos().rename('cos2')).addBands(timeRadians2.sin().rename('sin2'))

s2makeup = dailyMean.map(add_allbands)

def add_fittedNDRE(image):
  return image.addBands(image.select(harmonicIndependents).multiply(harmonicTrendCoefficients).reduce('sum').rename('NDRE'))

fitteds2 = s2makeup.map(add_fittedNDRE)

HANTS = fitteds2.select('NDRE')

In [19]:
# NDRE时间序列合成
image_agrgt = HANTS.toBands()
print(image_agrgt.bandNames().getInfo())

['0_NDRE', '1_NDRE', '2_NDRE', '3_NDRE', '4_NDRE', '5_NDRE', '6_NDRE', '7_NDRE', '8_NDRE', '9_NDRE', '10_NDRE', '11_NDRE', '12_NDRE', '13_NDRE', '14_NDRE', '15_NDRE', '16_NDRE', '17_NDRE', '18_NDRE', '19_NDRE', '20_NDRE', '21_NDRE', '22_NDRE', '23_NDRE', '24_NDRE']


In [20]:
image_agrgt_clip = image_agrgt.clip(roi)

In [21]:
samples_datesets=image_agrgt_clip.sampleRegions(collection=point,properties=['id','landcover'],scale=10,tileScale=8)

In [22]:
cols = ['landcover','id','0_NDRE','1_NDRE','2_NDRE','3_NDRE','4_NDRE','5_NDRE','6_NDRE','7_NDRE','8_NDRE','9_NDRE','10_NDRE','11_NDRE','12_NDRE','13_NDRE','14_NDRE','15_NDRE','16_NDRE','17_NDRE','18_NDRE','19_NDRE','20_NDRE','21_NDRE','22_NDRE','23_NDRE','24_NDRE']
inds = range(samples_datesets.size().getInfo())

In [23]:
df = pd.DataFrame(index=inds,columns=cols)
df.head()

,landcover,id,0_NDRE,1_NDRE,2_NDRE,3_NDRE,4_NDRE,5_NDRE,6_NDRE,7_NDRE,...,15_NDRE,16_NDRE,17_NDRE,18_NDRE,19_NDRE,20_NDRE,21_NDRE,22_NDRE,23_NDRE,24_NDRE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
data_value = samples_datesets.reduceColumns(
    reducer = ee.Reducer.toList().repeat(27),
    selectors = cols
)
df = pd.DataFrame(data_value.getInfo()['list'],cols)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,237,238,239,240,241,242,243,244,245,246
landcover,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
id,52.000000,54.000000,55.000000,56.000000,57.000000,58.000000,59.000000,61.000000,62.000000,63.000000,...,42.000000,43.000000,44.000000,45.000000,46.000000,47.000000,48.000000,49.000000,50.000000,51.000000
0_NDRE,0.258951,0.365595,0.383732,0.307149,0.228938,0.235850,0.250379,0.421522,0.465531,0.203679,...,0.309386,0.238959,0.226535,0.339769,0.277559,0.307612,0.111290,0.208738,0.304878,0.242161
1_NDRE,0.253517,0.361814,0.371660,0.299048,0.221335,0.234324,0.251745,0.406948,0.449178,0.178215,...,0.329151,0.222152,0.231088,0.319363,0.263021,0.302922,0.139395,0.206152,0.289536,0.242668
2_NDRE,0.253347,0.362759,0.364651,0.296960,0.217563,0.236626,0.255908,0.396370,0.435738,0.159512,...,0.349361,0.208738,0.236650,0.302757,0.251897,0.301040,0.170035,0.206882,0.278743,0.246263


In [25]:
df.to_csv('sample_2023-5dNDRE时间序列.csv',encoding='utf-8')